In [0]:
%pip install azure-storage-blob

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
storage_account_name = "ecom1sa"
container_name = "landing-2"
account_key = "XYX"

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", account_key)

# Access data
data_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/buyers-raw-2/Buyers-repartition-by-country.parquet"
df = spark.read.parquet(data_path)
df.show()

+-----------+------+---------+-------------+------------+----------+---------------+-------------+-----------------+--------------------+----------------------+------------------+-------------------------+---------------------+-------------------+-------------------+------------------+----------------------+----------------------+---------------------+------------------+------------------+-----------------+---------------------+---------------------+--------------------+---------------+------------------+-------------+-------------+----------------+----------------+
|    country|buyers|topbuyers|topbuyerratio|femalebuyers|malebuyers|topfemalebuyers|topmalebuyers|femalebuyersratio|topfemalebuyersratio|boughtperwishlistratio|boughtperlikeratio|topboughtperwishlistratio|topboughtperlikeratio|totalproductsbought|totalproductswished|totalproductsliked|toptotalproductsbought|toptotalproductswished|toptotalproductsliked|meanproductsbought|meanproductswished|meanproductsliked|topmeanproductsbo

In [0]:
storage_account_name = "ecom1sa"
container_name = "landing-2"
account_key = "XYX"

# Set Spark configuration
configs = {
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": account_key
}

# Mount the ADLS Gen2 location
mount_point = "/mnt/ecom_mnt"

dbutils.fs.mount(
    #source=f"wasbs://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    mount_point=mount_point,
    extra_configs=configs
)

# Verify Mount
display(dbutils.fs.ls(mount_point))

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3020595809201529>, line 13
     10 # Mount the ADLS Gen2 location
     11 mount_point = "/mnt/ecom_mnt"
---> 13 dbutils.fs.mount(
     14     #source=f"wasbs://{container_name}@{storage_account_name}.dfs.core.windows.net/",
     15     source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
     16     mount_point=mount_point,
     17     extra_configs=configs
     18 )
     20 # Verify Mount
     21 display(dbutils.fs.ls(mount_point))

File /databricks/python_shell/dbruntime/dbutils.py:154, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    152 exc.__context__ = None
    153 exc.__cause__ = None
--> 154 raise exc

ExecutionError: An error occurred while calling o416.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Direct

In [0]:
%fs ls "/mnt/ecom_mnt"

path,name,size,modificationTime
dbfs:/mnt/ecom_mnt/buyers-raw-2/,buyers-raw-2/,0,0
dbfs:/mnt/ecom_mnt/countries-raw-2/,countries-raw-2/,0,0
dbfs:/mnt/ecom_mnt/sellers-raw-2/,sellers-raw-2/,0,0
dbfs:/mnt/ecom_mnt/users-raw-2/,users-raw-2/,0,0


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()
spark

In [0]:
%fs ls "/mnt/ecom_mnt/users-raw-2"

path,name,size,modificationTime
dbfs:/mnt/ecom_mnt/users-raw-2/user_data_1.parquet,user_data_1.parquet,731432,1733891617000


In [0]:
user_df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("dbfs:/mnt/ecom_mnt/users-raw-2/")
user_df.show(1)

+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----------------+-----------+
|      identifierHash|type|   country|language|socialNbFollowers|socialNbFollows|socialProductsLiked|productsListed|productsSold|productsPassRate|productsWished|productsBought|gender|civilityGenderId|civilityTitle|hasAnyApp|hasAndroidApp|hasIosApp|hasProfilePicture|daysSinceLastLogin|seniority|seniorityAsMonths|seniorityAsYears|countryCode|
+--------------------+----+----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+-

In [0]:
user_df.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/users")

In [0]:
dbutils.fs.ls("/mnt/delta/tables/bronze/users")  # checking delta log for table creation

[FileInfo(path='dbfs:/mnt/delta/tables/bronze/users/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/delta/tables/bronze/users/part-00000-5d77d141-12a2-49ab-9a0e-ec44999933db-c000.snappy.parquet', name='part-00000-5d77d141-12a2-49ab-9a0e-ec44999933db-c000.snappy.parquet', size=480356, modificationTime=1733891809000),
 FileInfo(path='dbfs:/mnt/delta/tables/bronze/users/part-00000-9ff02ae5-470d-4464-9d25-750a0bd9138a-c000.snappy.parquet', name='part-00000-9ff02ae5-470d-4464-9d25-750a0bd9138a-c000.snappy.parquet', size=480356, modificationTime=1733960109000)]

In [0]:
buyer_df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("dbfs:/mnt/ecom_mnt/buyers-raw-2/")
buyer_df.show(2)

+-----------+------+---------+-------------+------------+----------+---------------+-------------+-----------------+--------------------+----------------------+------------------+-------------------------+---------------------+-------------------+-------------------+------------------+----------------------+----------------------+---------------------+------------------+------------------+-----------------+---------------------+---------------------+--------------------+---------------+------------------+-------------+-------------+----------------+----------------+
|    country|buyers|topbuyers|topbuyerratio|femalebuyers|malebuyers|topfemalebuyers|topmalebuyers|femalebuyersratio|topfemalebuyersratio|boughtperwishlistratio|boughtperlikeratio|topboughtperwishlistratio|topboughtperlikeratio|totalproductsbought|totalproductswished|totalproductsliked|toptotalproductsbought|toptotalproductswished|toptotalproductsliked|meanproductsbought|meanproductswished|meanproductsliked|topmeanproductsbo

In [0]:
buyer_df.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/buyers")

In [0]:
seller_df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("dbfs:/mnt/ecom_mnt/sellers-raw-2/")
#seller_df.show(2)
seller_df.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/sellers")

country_df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("dbfs:/mnt/ecom_mnt/countries-raw-2/")
#country_df.show(2)
country_df.write.format("delta")\
    .mode("overwrite")\
    .save("/mnt/delta/tables/bronze/countries")



In [0]:
%fs ls  "/mnt/delta/tables/bronze/users"

path,name,size,modificationTime
dbfs:/mnt/delta/tables/bronze/users/_delta_log/,_delta_log/,0,0
dbfs:/mnt/delta/tables/bronze/users/part-00000-5d77d141-12a2-49ab-9a0e-ec44999933db-c000.snappy.parquet,part-00000-5d77d141-12a2-49ab-9a0e-ec44999933db-c000.snappy.parquet,480356,1733891809000
dbfs:/mnt/delta/tables/bronze/users/part-00000-9ff02ae5-470d-4464-9d25-750a0bd9138a-c000.snappy.parquet,part-00000-9ff02ae5-470d-4464-9d25-750a0bd9138a-c000.snappy.parquet,480356,1733960109000
